In [1]:
delete from [Theme_Park].[Maintenance_Calendar]
go
drop table [Theme_Park].[Maintenance_Notification]
go
create table [Theme_Park].[Maintenance_Notification]
(
    notification_id int identity(1,1),
    maintenance_id int not null,
    completion datetime not null,
    billed_hours int,
    amount smallmoney,
)

: Msg 3701, Level 11, State 5, Line 1
Cannot drop the table 'Theme_Park.Maintenance_Notification', because it does not exist or you do not have permission.

Commands completed successfully.

Total execution time: 00:00:00.121

## Trigger TRIGGER_MANAGER_NOTIFICATION_ON_MAINTENANCE_COMPLETION:

This trigger is intended to create notification entries whenever a maintenance event is updated or inserted
with a completion time set. These notifications have 


In [1]:
drop trigger [Theme_Park].[Trigger_Manager_Notification_On_Maintenance_Completion]
go
create trigger [Theme_Park].[Trigger_Manager_Notification_On_Maintenance_Completion]
on [Theme_Park].[Maintenance_Calendar]
after insert,update
as
BEGIN
    set nocount on
    declare 
        @id int,
        @Completion_Date datetime,
        @Hours int,
        @Amount smallmoney,
        @Description varchar(50)
    declare cur cursor read_only for 
        select Maint_ID, Maint_Completion, Billed_Hours, Invoice_amount, Maint_Description from inserted
    open cur
    fetch next from cur into @id, @Completion_Date, @Hours, @Amount, @Description
    while @@fetch_status = 0
    begin
        if (@Completion_Date is not null)
        begin
            insert into [Theme_Park].[Maintenance_Notification]
            values (@id, @Completion_Date, @Hours, @Amount, @Description)
        end
        fetch next from cur into @id, @Completion_Date, @Hours, @Amount, @Description
    end
    close cur
    deallocate cur
end
    

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.112

## Trigger TRIGGER_MANAGER_NOTIFICATION_ON_EMPLOYEE_CHANGE:

This trigger fires whenever an employee's pay or department changes

In [ ]:
drop trigger [Theme_Park].[Trigger_Manager_Notification_On_Employee_Change]
go
create trigger [Theme_Park].[Trigger_Manager_Notification_On_Employee_Change]
on [Theme_Park].[Employee]
after update, insert
as
BEGIN
    set nocount on
    declare 
        @fname varchar(20),
        @lname varchar(20),
        @ssn numeric(9,0),
        @role varchar(20),
        @dept_id int,
        @payrate money,
        @supervisor_ssn int,
        @salaried bit
    -- Here we use COLUMNS_UPDATED to check for the updated columns of the rows
    -- This goes from right to left, as a bitmask. A 1 in the nth-most bit from the right, means that column n of [Theme_Park].[Employee] was updated
    -- Payrate is column 12, salaried is column 11, department is column 8, role is column 9
    declare @payrate_bitmask varbinary = LEFT_SHIFT(1, 11),
            @salaried_bitmask varbinary = LEFT_SHIFT(1, 10),
            @department_bitmask varbinary = LEFT_SHIFT(1, 7),
            @role_bitmask varbinary = LEFT_SHIFT(1, 8)
    declare cur cursor read_only for 
        select fname, lname, ssn, dept_id, role, payrate, salaried, supervisor_ssn from inserted
    open cur
    fetch next from cur into @fname, @lname, @ssn, @dept_id, @role, @payrate, @salaried, @supervisor_ssn
    while @@fetch_status = 0
    begin
        declare @updated_columns varbinary = COLUMNs_UPDATED()
        declare @dept_head_ssn int = (select supervisor_ssn from [Theme_Park].[Department] where dept_id = @dept_id)
        declare @super_fname varchar(20), @super_lname varchar(20) = (select fname, lname from [Theme_Park].[Employees] where ssn = @supervisor_ssn)
        if (@updated_columns & (@payrate_bitmask + @salaried_bitmask) <> 0)
        begin
            insert into [Theme_Park].[Manager_Notifications]
            values 
            (@supervisor_ssn, @fname, @lname, @dept_id, @role, @payrate, @salaried, @super_fname, @super_lname, 'P'),
            (@dept_head, @fname, @lname, @dept_id, @role, @payrate, @salaried, @super_fname, @super_lname, 'P')
        end
        if (@updated_columns & (@department_bitmask + @role_bitmask) <> 0)
        begin
            insert into [Theme_Park].[Manager_Notifications]
            values 
            (@supervisor_ssn, @fname, @lname, @dept_id, @role, @payrate, @salaried, @super_fname, @super_lname, 'R'),
            (@dept_head, @fname, @lname, @dept_id, @role, @payrate, @salaried, @super_fname, @super_lname, 'R')        
        end
        fetch next from cur into @fname, @lname, @ssn, @dept_id, @role, @payrate, @salaried
    end
    close cur
    deallocate cur
end
    

In [1]:
drop table [Theme_Park].[Manager_Notifications]
go
create table [Theme_Park].[Manager_Notifications]
(
    manager_ssn int not null,
    employee_fname varchar(20),
    employee_lname varchar(20),
    dept_id int,
    role varchar(20),
    payrate money,
    salaried bit,
    super_fname varchar(20),
    super_lname varchar(20),
    type char not null
)

: Msg 3701, Level 11, State 5, Line 1
Cannot drop the table 'Theme_Park.Manager_Notifications', because it does not exist or you do not have permission.

Commands completed successfully.

Total execution time: 00:00:00.119

In [ ]:
drop proc [Theme_Park].[Proc_Get_Manager_Notifications]


In [4]:
insert into [Theme_Park].[Maintenance_Calendar]
(Occurence_datetime, Priority, Maint_Start, Maint_Completion, Billed_Hours, Invoice_amount)
values
(
    GETDATE(), 'ASAP', GETDATE(), GETDATE(), 1, 1130.50 
)

(1 row affected)

Total execution time: 00:00:00.061

In [8]:
insert into [Theme_Park].[Maintenance_Calendar]
(Occurence_datetime, Priority, Maint_Start)
values
(
    GETDATE(), 'ASAP', GETDATE()
)

(1 row affected)

Total execution time: 00:00:00.053

In [12]:
update [Theme_Park].[Maintenance_Calendar]
set Maint_Completion = GETDATE()
where Maint_ID = 3
go

(1 row affected)

Total execution time: 00:00:00.058

In [2]:
select * from [Theme_Park].[Maintenance_Notification]

(3 rows affected)

Total execution time: 00:00:00.214

notification_id,maintenance_id,completion,billed_hours,amount,maint_description
1,1,2022-11-11 00:00:00.000,2,1.00,Broken wheel
2,3,2022-11-10 00:00:00.000,2,1.00,jflj
3,2,2022-11-10 00:00:00.000,2,420.22,kdkdkd


In [16]:
declare @date datetime = GETDATE()
exec [Theme_Park].[Proc_Add_Maint] @date, null, null, 'Urgent', @date
go 

(1 row affected)

Total execution time: 00:00:00.086

In [4]:
select * from [Theme_Park].[Ticket_Reservation]
go
select * from [Theme_Park].[Maintenance_Calendar]
go
select * from [Theme_Park].[Attractions]
GO
select * from [Theme_Park].[Vendor]
go
select * from [Theme_Park].[Department]
go
select * from [Theme_Park].[Ticket]
go
select * from [Theme_Park].[Merchandise_Sale]
go
select * from [Theme_Park].[Merchandise]
go
select * from [Theme_Park].[Employee]
go

(1 row affected)

(4 rows affected)

(3 rows affected)

(2 rows affected)

(2 rows affected)

...

dept_id,dept_name,supervisor_ssn
1,Rides,NULL
2,Food Court,NULL


Ticket_ID,Date,Ticket_Class,Price,Reservation


transaction_id,item_id,quantity,sale_price,attraction_id,transaction_date


item_id,name,price,merch_type


fname,lname,ssn,gender,address,phone,date_joined,dept_id,role,supervisor_ssn,salaried,payrate,vacation_days


In [5]:
alter table [Theme_Park].[Ticket_Reservation]
alter column Credit_Card_Number bigint
go

Commands completed successfully.

Total execution time: 00:00:00.096

In [ ]:
exec [Theme_Park].[Proc_Customer_Buy_Ticket] GETDATE(), 'Poor', 